In [53]:
import csv

characters = []

with open("HP_characters.csv", "r", encoding="utf8") as sent_file:
    csv = csv.reader(sent_file, delimiter=",")
    for row in csv:
        name = row[0].replace(' ', '_')
        parentage = row[1]
        house = row[2]
        occupation = row[3]
        characters.append((name, parentage, house, occupation))

#characters

In [54]:
# Try to get the pages:
import urllib.request
import json
from urllib.parse import quote

url = 'http://example.com/'
response = urllib.request.urlopen(url)
data = response.read()      # a `bytes` object
text = data.decode('utf-8')

misses = []
lengths = []
disambig = []

#baseurl = "https://en.wikipedia.org/w/api.php?"

#https://harrypotter.fandom.com/wiki/Special:Search?query=Dumbledore&scope=internal&navigationSearch=true&so=trending
#https://www.reddit.com/r/learnpython/comments/muwu7v/scraping_fandomwiki_pages/
baseurl = "https://harrypotter.fandom.com/api.php?"

action = "action=query"

for character in characters:
    title = "titles=" + character[0]
    #print(title)
    content = "prop=revisions&rvprop=content"
    dataformat ="format=json"
    query = "{}{}&{}&{}&{}".format(baseurl, action, content, title, dataformat)
    #print(query)
    #try:
    wikiresponse = urllib.request.urlopen(query)
    wikidata = wikiresponse.read()
    wikitext = wikidata.decode('utf-8')
    lengths.append((character, len(wikitext)))

    #print(wikitext)
    try:
        jsonobj = json.loads(wikitext)
    except ValueError: 
        print(f'Decoding JSON has failed for {character}, moving on...')
    num = list(jsonobj['query']['pages'].keys())[0]
    wikitext = jsonobj['query']['pages'][num]['revisions'][0]['*']
    #wikitext = jsonobj['query']['pages'][num]['revisions'][0]['slots']['main']['*']
    f_name = character[0] + '.txt'
    with open("characters/" + f_name, 'w') as f:
        f.write(wikitext)


    #print(len(wikitext))
      #except:
  #  misses.append(quote(character))

In [29]:

print(sorted(lengths, key=lambda x: x[1]))

[(('Hetty_Bayliss', 'Muggle', 'other', 'other'), 1856), (('Nick_Clegg', 'Muggle', 'other', 'other'), 2191), (('Cook_of_The_Hanged_Man', 'Muggle', 'other', 'other'), 2316), (('Rosemary_Ames', 'Muggle', 'other', 'other'), 2331), (("Seamus_Finnigan's_father", 'Muggle', 'other', 'other'), 2331), (("Creevey_brothers'_father", 'Muggle', 'other', 'other'), 2578), (('Gordon', 'Muggle', 'other', 'other'), 2616), (('Yvette_Coquelot', 'Muggle', 'other', 'other'), 2832), (('Dot', 'Muggle', 'other', 'other'), 2865), (('Dennis_Bishop', 'Muggle', 'other', 'other'), 2996), (('Amy_Benson', 'Muggle', 'other', 'other'), 3037), (('Dennis', 'Muggle', 'other', 'other'), 3056), (("Dudley_Dursley's_wife", 'Muggle', 'other', 'other'), 3095), (('Herbert_Chorley', 'Muggle', 'other', 'other'), 3167), (("Mary_Cattermole's_mother", 'Muggle', 'other', 'other'), 3218), (('Morag_MacDougal', 'Pure-Blood', 'Ravenclaw', 'Student'), 3313), (('Lestrange', 'Pure-Blood', 'Slytherin', 'Other'), 3431), (("Dudley_Dursley's_chil

In [50]:
import os

unzipped = list(list(zip(*characters))[0])
names = []

for file in os.listdir("characters"):
    names.append(file[:-4])

print(len(names))
print(len(unzipped))    
i = 1
for name in unzipped:
    if name in names:
        print(f"{i} {name}")
        i = i + 1
        
print(len(list(set(unzipped))))

124
125
1 Hannah_Abbott
2 Susan_Bones
3 Mandy_Brocklehurst
4 Millicent_Bulstrode
5 Aberforth_Dumbledore
6 Albus_Dumbledore
7 Seamus_Finnigan
8 Mundungus_Fletcher
9 Anthony_Goldstein
10 Hugo_Granger-Weasley
11 Rose_Granger-Weasley
12 Gilderoy_Lockhart
13 Remus_Lupin
14 Edward_Lupin
15 Minerva_McGonagall
16 Garrick_Ollivander
17 James_Potter_II
18 Albus_Potter
19 Harry_Potter
20 Lily_J._Potter
21 Quirinus_Quirrell
22 Tom_Riddle
23 Severus_Snape
24 Dean_Thomas
25 Nymphadora_Tonks
26 Sybill_Trelawney
27 Dolores_Umbridge
28 Apolline_Delacour
29 Fleur_Delacour
30 Gabrielle_Delacour
31 Irma_Dugard
32 Filius_Flitwick
33 Rubeus_Hagrid
34 Olympe_Maxime
35 Mary_Cattermole
36 Colin_Creevey
37 Dennis_Creevey
38 Dirk_Cresswell
39 Kevin_Entwhistle
40 Justin_Finch-Fletchley
41 Hermione_Granger
42 Lily_J._Potter
43 Edward_Tonks
44 Donaghan_Tremlett
45 Myrtle_Warren
46 Rosemary_Ames
47 Hetty_Bayliss
48 Amy_Benson
49 Dennis_Bishop
50 Frank_Bryce
51 Mary_Cattermole's_mother
52 Cecilia
53 Herbert_Chorley
5

In [52]:
for n in unzipped:
    print(n)

Hannah_Abbott
Susan_Bones
Mandy_Brocklehurst
Millicent_Bulstrode
Aberforth_Dumbledore
Albus_Dumbledore
Seamus_Finnigan
Mundungus_Fletcher
Anthony_Goldstein
Hugo_Granger-Weasley
Rose_Granger-Weasley
Gilderoy_Lockhart
Remus_Lupin
Edward_Lupin
Minerva_McGonagall
Garrick_Ollivander
James_Potter_II
Albus_Potter
Harry_Potter
Lily_J._Potter
Quirinus_Quirrell
Tom_Riddle
Severus_Snape
Dean_Thomas
Nymphadora_Tonks
Sybill_Trelawney
Dolores_Umbridge
Apolline_Delacour
Fleur_Delacour
Gabrielle_Delacour
Irma_Dugard
Filius_Flitwick
Rubeus_Hagrid
Olympe_Maxime
Mary_Cattermole
Colin_Creevey
Dennis_Creevey
Dirk_Cresswell
Kevin_Entwhistle
Justin_Finch-Fletchley
Hermione_Granger
Lily_J._Potter
Edward_Tonks
Donaghan_Tremlett
Myrtle_Warren
Rosemary_Ames
Hetty_Bayliss
Amy_Benson
Dennis_Bishop
Frank_Bryce
Mary_Cattermole's_mother
Cecilia
Herbert_Chorley
Nick_Clegg
Cole
Cook_of_The_Hanged_Man
Yvette_Coquelot
Creevey_brothers'_father
Dennis
Dot
Dudley_Dursley's_children
Dudley_Dursley
Marjorie_Dursley
Dudley_Dur